In [1]:
import sqlite3

In [2]:
import pandas as pd

In [3]:
conn = sqlite3.connect("db/pharm_data.db", check_same_thread=False)

In [22]:
global_vars={}
global_vars['prc'] = '4'
global_vars['meds'] = map(str,[197319,
1049221,
835603,
313988,
859751,
896185])

out = pd.read_sql("select \
                PLAN_INFO.CONTRACT_NAME, \
                PLAN_INFO.PLAN_NAME, \
                PLAN_INFO.CONTRACT_ID, \
                PLAN_INFO.PLAN_ID, \
                PLAN_INFO.SEGMENT_ID, \
                PLAN_INFO.FORMULARY_ID, \
                PLAN_INFO.PREMIUM, \
                PLAN_INFO.DEDUCTIBLE \
        from \
            FORMULARY_DATA join PLAN_INFO \
            on substr(FORMULARY_DATA.FORMULARY_ID, 4) = PLAN_INFO.FORMULARY_ID \
        where \
              PLAN_INFO.PDP_REGION_CODE is '"+str(global_vars['prc'])+"' and \
              PLAN_INFO.CONTRACT_ID like 'S%'\
        order by PLAN_INFO.FORMULARY_ID",conn)\
.drop_duplicates(subset='FORMULARY_ID',keep='first')\
.reset_index(drop=True)
for med in global_vars['meds']:
        out = pd.concat([out,
           pd.read_sql(" ".join(("select \
                    PLAN_INFO.CONTRACT_NAME, \
                    PLAN_INFO.PLAN_NAME, \
                    PLAN_INFO.CONTRACT_ID, \
                    PLAN_INFO.PLAN_ID, \
                    PLAN_INFO.SEGMENT_ID, \
                    PLAN_INFO.FORMULARY_ID, \
                    PLAN_INFO.PREMIUM, \
                    PLAN_INFO.DEDUCTIBLE, \
                    FORMULARY_DATA.RXCUI as MED_"+med+", \
                    FORMULARY_DATA.PRIOR_AUTHORIZATION_YN as PA_"+med+", \
                    FORMULARY_DATA.TIER_LEVEL_VALUE as TIER_"+med+", \
                    BENEFICIARY_COST.COST_AMT_PREF as CAP_"+med+" \
            from \
                FORMULARY_DATA join PLAN_INFO \
                    on substr(FORMULARY_DATA.FORMULARY_ID, 4) = PLAN_INFO.FORMULARY_ID \
                join BENEFICIARY_COST \
                    on PLAN_INFO.CONTRACT_ID = BENEFICIARY_COST.CONTRACT_ID and \
                       PLAN_INFO.PLAN_ID = BENEFICIARY_COST.PLAN_ID and \
                       cast(FORMULARY_DATA.TIER_LEVEL_VALUE as integer) = \
                                                cast(BENEFICIARY_COST.TIER as integer) \
            where \
                  PLAN_INFO.PDP_REGION_CODE is '"+str(global_vars['prc'])+"' and \
                  FORMULARY_DATA.RXCUI like '%"+med+"' and \
                  PLAN_INFO.CONTRACT_ID like 'S%' and \
                  (cast(BENEFICIARY_COST.COVERAGE_LEVEL as integer) = 0 or \
                  cast(BENEFICIARY_COST.COVERAGE_LEVEL as integer) = 1) and \
                  BENEFICIARY_COST.DAYS_SUPPLY = 1 and \
                  cast(BENEFICIARY_COST.COST_TYPE_PREF as integer) = 1\
            order by PLAN_INFO.FORMULARY_ID").split()),conn)\
        .drop_duplicates(subset='FORMULARY_ID',keep='first')\
        .reset_index(drop=True)\
        [['MED_'+med,
          'PA_'+med,
          'TIER_'+med,
          'CAP_'+med]]],axis=1)
#ADD IN COST COLUMN
        if 'ANNUAL_TOTAL_COST' in out.columns:
            del out['ANNUAL_TOTAL_COST']
        if 'MONTHLY_COPAY' in out.columns:
            del out['MONTHLY_COPAY']
        out['ANNUAL_TOTAL_COST'] = (out[[c for c in out.columns if c[:3] == 'CAP']]*12).sum(axis=1) + out['PREMIUM']*12 + out['DEDUCTIBLE']
        out['MONTHLY_COPAY'] = out[[c for c in out.columns if c[:3] == 'CAP']].sum(axis=1)
        out = out.sort_values(by='ANNUAL_TOTAL_COST').reset_index(drop=True).dropna()

display(out)

,CONTRACT_NAME,PLAN_NAME,CONTRACT_ID,PLAN_ID,SEGMENT_ID,FORMULARY_ID,PREMIUM,DEDUCTIBLE,MED_197319,PA_197319,...,MED_859751,PA_859751,TIER_859751,CAP_859751,MED_896185,PA_896185,TIER_896185,CAP_896185,ANNUAL_TOTAL_COST,MONTHLY_COPAY
0,"SYMPHONIX HEALTH INSURANCE, INC.",Symphonix Value Rx (PDP),S0522,4.0,0.0,18041,30.4,405.0,197319,N,...,859751,N,2,6.0,896185,N,3.0,30.0,1393.8,52.0
1,UNITEDHEALTHCARE INS. CO. AND UNITEDHEALTHCARE NY,AARP MedicareRx Saver Plus (PDP),S5921,349.0,0.0,18040,33.2,405.0,197319,N,...,859751,N,2,3.0,896185,N,3.0,37.0,1475.4,56.0
2,CIGNA HEALTH AND LIFE INSURANCE COMPANY,Cigna-HealthSpring Rx Secure-Extra (PDP),S5617,249.0,0.0,18083,52.9,0.0,197319,N,...,859751,N,2,5.0,896185,N,3.0,30.0,1546.8,76.0
3,MEDCO CONTAINMENT LIFE AND MEDCO CONTAINMENT NY,Express Scripts Medicare - Value (PDP),S5660,106.0,0.0,18154,34.0,405.0,197319,N,...,859751,N,2,10.0,896185,N,3.0,47.0,1641.0,69.0
4,SILVERSCRIPT INSURANCE COMPANY,SilverScript Choice (PDP),S5601,8.0,0.0,18419,34.3,0.0,197319,N,...,859751,N,2,6.0,896185,N,3.0,47.0,1695.6,107.0
5,MEDCO CONTAINMENT LIFE AND MEDCO CONTAINMENT NY,Express Scripts Medicare - Saver (PDP),S5660,220.0,0.0,18152,22.6,405.0,197319,N,...,859751,N,2,8.0,896185,N,3.0,34.0,1756.2,90.0
6,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Saver (PDP),S5810,38.0,0.0,18056,34.9,325.0,197319,N,...,859751,N,2,8.0,896185,N,3.0,32.0,1799.8,88.0
7,UNITEDHEALTHCARE INS. CO. AND UNITEDHEALTHCARE NY,AARP MedicareRx Walgreens (PDP),S5921,386.0,0.0,18043,26.8,405.0,197319,N,...,859751,N,2,10.0,896185,N,3.0,42.0,1806.6,90.0
9,HUMANA INSURANCE COMPANY,Humana Enhanced (PDP),S5884,62.0,0.0,18256,77.8,0.0,197319,N,...,859751,N,2,1.0,896185,N,3.0,42.0,1977.6,87.0
10,HUMANA INSURANCE COMPANY,Humana Walmart Rx Plan (PDP),S5884,150.0,0.0,18255,20.4,405.0,197319,N,...,859751,N,2,4.0,896185,N,3.0,42.0,2041.8,116.0
